# InvestBot: a bot for stock classification using fundamentalist analysis

InvestBot is a decision support tool for investors. It consists of classifying stocks on the stock exchange as a reliable investment or not, based on the following criteria:

- if the stock price will yield until the next quarter

- if the stock price will yield more than the quote of the ibovespa index until the next quarter

That is, a stock will be classified as reliable if it yields more than the ibovespa index. If the stock depreciates, regardless of whether it continues to be higher than the Ibovespa, it will be classified as unreliable, after all, no one wants to invest in a stock that will depreciate, right?

**But the question that doesn't want to be silent is: how will we predict if a stock will appreciate until the next quarter?**

Basically, we will use a very famous approach in the investment world: fundamental analysis. This technique aims to determine the intrinsic value of a company, and its growth potential. For this, it uses some data that are called fundamental indicators, and these can be found from the balance sheet and income statement of companies.

With the data from the indicators collected, and the classes of stocks already defined, following the criteria explained above, some classification models will be trained, in order that they can predict whether a stock will appreciate more than the ibovespa index until the next quarter.

However, a doubt may arise: **fundamental analysis is used for medium and long-term investments, so is it possible to analyze the data of fundamental indicators to determine if a stock will appreciate until the next quarter? Well, that's what we'll find out at the end of this work.** As the fundamental analysis makes it possible to discover the real value of a stock, it may be possible from it to determine whether a stock will appreciate in value until the next quarter.

**The main objective here is to build a tool that supports investors' decision making for short-term investments.**

In the next sections, each of the steps necessary for us to reach the desired result will be developed.

## Solution Proposal

To solve the problem, we will follow these steps:

- **0.0.** Data Collection.
- **1.0.** Data description.
- **2.0.** Feature Engineering
- **3.0.** Filtering the features
- **4.0.** Exploratory Data Analysis
- **5.0.** Data preparation
- **6.0.** Feature selection
- **7.0.** Machine Learning Modelling
- **8.0.** Hyperparameter fine tuning
- **9.0.** Machine Learning Performance
- **10.0.** Deploy Model to Production

<p align='center'>
    <img src='img/bot.jpg'>
</p>

# 0.0 Imports

In [7]:
import pandas                      as pd
import numpy                       as np
import matplotlib.pyplot           as plt
import seaborn                     as sns
import os

from pandas_datareader import data as web

import warnings
warnings.filterwarnings("ignore")

## 0.1. Helper Functions

in this step, we'll document all the functions that will be used to solve the problem.

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
def collect_balances( fundamentals, files, companies ):
    
    for file in files:
        # collecting only the company name
        name = file[:-4]

        # Balance Sheet
        balance = pd.read_excel( f'Balances/{file}', sheet_name=0 )

        # Name of the company as a title in the first column
        balance.iloc[0, 0] = name

        # Make the first line a header
        balance.columns = balance.iloc[0]
        balance = balance.iloc[1:]

        # Make the first column an index
        balance = balance.set_index( name )

        # Cash Flow Statement
        cfs = pd.read_excel( f'Balances/{file}', sheet_name=1 )

        # Name of the company as a title in the first column
        cfs.iloc[0, 0] = name

        # Make the first line a header
        cfs.columns = cfs.iloc[0]
        cfs = cfs.iloc[1:]

        # Make the first column an index
        cfs = cfs.set_index( name )

        try:
            fundamentals[name] = balance.append( cfs )
            companies.append( name )
        except:
            pass
        
def read_quotes( quotes ):
    
    # getting the file name
    files = os.listdir( "Quotes" )
    
    for file in files:
        name = file[:-4]
        quotes[name] = pd.read_csv( f'Quotes/{file}' ) 
        
def collect_ibov_quotes( fundamentals ):
    
    # time course
    initial_date = "06/30/2011"
    final_date = "03/31/2020"

    df_ibov = web.DataReader( '^BVSP', data_source='yahoo', start=initial_date, end=final_date )

    # Adding dates that don't exist in df_ibov
    dates = fundamentals['PETR4'].index
    for date in dates:
        if date not in df_ibov.index:
            # new dates receiving empty values
            df_ibov.loc[date] = np.nan 
            
    # sorting df by the indexes
    df_ibov = df_ibov.sort_index()
    # filling empty lines by the values from above
    df_ibov = df_ibov.ffill()
    # rename column adj close
    df_ibov = df_ibov.rename( columns={'Adj Close': 'IBOV'} )
    
    return df_ibov

In [3]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## 0.2. Loading Data

The data needed to build our InvestBot bot are as follows:

- Balance Sheet and Cash Flow Statement
- Stock prices
- Quotes of the Ibovespa index

**1.** The balance sheets and cash flow statements of the companies will be used to calculate the fundamental indicators of the companies. They were previously collected on the website [Fundamentus](https://www.fundamentus.com.br/resultado.php), And they are in the folder "/Balances". From the data found in these spreadsheets, it will be possible to calculate the fundamental indicators of the companies.

**2.** The collection of quote data was done through the "pandas_datareader" library, where it searches for quotes on the [Yahoo Finances](https://br.financas.yahoo.com/) website. The period analyzed will be from 06/30/2011 to 03/31/2020.

**3.** Similarly, Ibovespa quotations were also collected in the same period mentioned above.

### 0.2.1. Balance Sheet and Cash Flow Statement

As stated earlier, the quote data was collected manually and placed in the "/Balances" folder.

In [53]:
# Collect balance sheet and cash flow statement
fundamentals = {}
files = os.listdir( "Balances" )
companies = []

collect_balances( fundamentals, files, companies )

WARNING *** file size (39707) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (39707) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (67870) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (67870) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (67870) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (67870) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (55799) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** file size (64198) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64198) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64198) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64198) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64198) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64200) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64200) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** file size (53974) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64194) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64194) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64198) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64198) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (18233) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (18233) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (16355) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (57152) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (57152) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (57152) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (57152) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (70008) not 512 + multiple of sector size (5

WARNING *** file size (34345) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (34345) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (28679) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (28679) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (63804) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (63804) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (63804) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

...

WARNING *** file size (69990) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69990) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69990) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69990) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69990) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69990) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (25444) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** file size (64202) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64202) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64202) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64202) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64202) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64202) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (70028) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** file size (11846) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69994) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69994) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69994) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (69994) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64192) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64192) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** file size (64196) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64196) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64196) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (53948) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (53948) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (53948) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (53948) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

WARNING *** file size (70006) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64208) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64208) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64208) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64208) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64186) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (64186) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero

### 0.2.2. Stock Quotes

The first time, the quotes were collected and placed in the "/Quotes" folder as .csv files. With that, the "read_quotes" function was created in order to read the quotes from the folder.

In [ ]:
# Collect stock quotes
quotes = {}

for company in companies:
    try:
        quotes[company] = web.DataReader( f'{company}.SA', data_source='yahoo', start="06/30/2011", end="03/31/2020" )
    except:
        pass

In [20]:
# Saving quotes
for company in quotes:
    quotes[company].to_csv( f'Quotes/{company}.csv' )

In [54]:
# Read stock quotes
quotes = {}
read_quotes( quotes )

### 0.2.2. Remove companies from the fundamentals that do not have quotes for the period collected

In [55]:
for company in companies:
    if company not in quotes:
        fundamentals.pop( company )

companies = list( quotes.keys() )

### 0.2.3. Remove companies that have empty quotes

In [56]:
for company in companies:
    if quotes[company].isnull().values.any():
        quotes.pop( company )
        fundamentals.pop( company )

companies = list( quotes.keys() )

### 0.2.4. Adding dates that don't exist in the quotes 

In [69]:
# Adding dates for quarters that do not exist in quote dataframes
for company in fundamentals:
    # Putting quote date as dataframe index
    quotes[company] = quotes[company].set_index( "Date" )
    first_date = quotes[company].index[0].split( '-' )
    last_date = quotes[company].index[-1].split( '-' )
    
    for date in fundamentals[company].columns:
        try:
            date_aux = date.split( '/' )
        except:
            continue
        
        # Selecting only dates before 2020/04/01
        if datetime.date( int( date_aux[2] ), int( date_aux[1] ), int( date_aux[0] ) ) <= datetime.date( int( last_date[0] ), int( last_date[1] ), int( last_date[2] ) ) and datetime.date( int( date_aux[2] ), int( date_aux[1] ), int( date_aux[0] ) ) >= datetime.date( int( first_date[0] ), int( first_date[1] ), int( first_date[2] ) ):
            formatted_date = date_aux[2] + '-' + date_aux[1] + '-' + date_aux[0]
            # Inserting new lines with empty fields
            if formatted_date not in list( quotes[company].index ):
                quotes[company].loc[formatted_date] = np.nan

# Sorting the df and filling the created fields
for company in quotes:
    # sorting by date
    quotes[company] = quotes[company].sort_index()
    
    # filling empty fields with data from the previous row
    quotes[company] = quotes[company].ffill()


In [70]:
for c in quotes:
    if quotes[c]['Adj Close'].isnull().any():
        print(c)

### 0.2.4. Joining the fundamentals and the quotes

In [71]:
for company in fundamentals:
    # Fundamentals
    # Turn rows into columns
    table = fundamentals[company].T
    
    # Transform index to date type
    table.index = pd.to_datetime( table.index, format="%d/%m/%Y" )
    
    # Quotes
    quote_table = quotes[company]
    
    # Select only the column Adj Close
    quote_table = quote_table[['Adj Close']]
    
    # Merge
    table = table.merge( quote_table, right_index=True, left_index=True )
    
    # Inserting name in index column
    table.index.name = company
    
    fundamentals[company] = table

### 0.2.5. Treat columns

- Catch only companies that have the same columns
- Columns with repeated names
- Missing values

**1. Catch only companies that have the same columns**

In [72]:
columns = list ( fundamentals['PETR4'].columns )

for company in companies:
    # Checking if the company has the necessary columns
    if set( columns ) != set( fundamentals[company].columns ):
        fundamentals.pop( company )

**2. Columns with repeated names**

In [73]:
# Changing the name of columns that have the same names

text_columns = ";".join( columns )

modified_columns = []
for column in columns:
    if columns.count( column ) == 2 and column not in modified_columns:
        text_columns = text_columns.replace( ";" + column + ";", ";" + column + "_1;", 1 )
        modified_columns.append( column )
        
columns = text_columns.split( ';' )

In [74]:
# Implement columns in tables
for company in fundamentals:
    fundamentals[company].columns = columns

**3. Missing values**

In [75]:
# Counting the total number of missing values in each column

# Start dictionary with all keys equal to zero
nan_values = dict.fromkeys( columns, 0 )
total_lines = 0

for company in fundamentals:
    tabel = fundamentals[company]
    # Total rows to know the proportion of missing values in each column
    total_lines += tabel.shape[0]
    
    for column in columns:
        qnt_nan = pd.isnull( tabel[column] ).sum()
        # Dictionary with the amount of nan values in each column
        nan_values[column] += qnt_nan
        
print( nan_values )
print( total_lines )

{'Ativo Total': 0, 'Ativo Circulante': 0, 'Caixa e Equivalentes de Caixa': 0, 'Aplicações Financeiras': 0, 'Contas a Receber_1': 0, 'Estoques_1': 0, 'Ativos Biológicos_1': 0, 'Tributos a Recuperar': 0, 'Despesas Antecipadas_1': 0, 'Outros Ativos Circulantes': 0, 'Ativo Realizável a Longo Prazo': 0, 'Aplicações Financeiras Avaliadas a Valor Justo': 0, 'Aplicações Financeiras Avaliadas ao Custo Amortizado': 0, 'Contas a Receber': 0, 'Estoques': 0, 'Ativos Biológicos': 0, 'Tributos Diferidos_1': 0, 'Despesas Antecipadas': 0, 'Créditos com Partes Relacionadas': 0, 'Outros Ativos Não Circulantes': 0, 'Investimentos': 0, 'Imobilizado': 0, 'Intangível': 0, 'Diferido': 0, 'Passivo Total': 0, 'Passivo Circulante': 0, 'Obrigações Sociais e Trabalhistas': 0, 'Fornecedores': 0, 'Obrigações Fiscais': 0, 'Empréstimos e Financiamentos_1': 0, 'Passivos com Partes Relacionadas_1': 0, 'Dividendos e JCP a Pagar': 0, 'Outros_1': 0, 'Provisões_1': 0, 'Passivos sobre Ativos Não-Correntes a Venda e Descontin

In [49]:
# collect columns that will be removed
delete_columns = []

for column in nan_values:
    if nan_values[column] > 50:
        delete_columns.append( column )
        
for company in fundamentals:
    fundamentals[company] = fundamentals[company].drop( delete_columns, axis=1 )
    fundamentals[company] = fundamentals[company].ffill()

### 0.2.6. Creating labels

In [50]:
df_ibov = collect_ibov_quotes( fundamentals )

# putting ibov column in fundamentals tables
for company in fundamentals:
    fundamentals[company] = fundamentals[company].merge( df_ibov[['IBOV']], left_index=True, right_index=True )

fundamentals['PETR4'].head()

,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber_1,Estoques_1,Ativos Biológicos_1,Tributos a Recuperar,Despesas Antecipadas_1,Outros Ativos Circulantes,Ativo Realizável a Longo Prazo,Aplicações Financeiras Avaliadas a Valor Justo,Aplicações Financeiras Avaliadas ao Custo Amortizado,Contas a Receber,Estoques,Ativos Biológicos,Tributos Diferidos_1,Despesas Antecipadas,Créditos com Partes Relacionadas,Outros Ativos Não Circulantes,Investimentos,Imobilizado,Intangível,Diferido,Passivo Total,Passivo Circulante,Obrigações Sociais e Trabalhistas,Fornecedores,Obrigações Fiscais,Empréstimos e Financiamentos_1,Passivos com Partes Relacionadas_1,Dividendos e JCP a Pagar,Outros_1,Provisões_1,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados_1,Passivo Não Circulante,Empréstimos e Financiamentos,Passivos com Partes Relacionadas,Outros,Tributos Diferidos,Adiantamento para Futuro Aumento Capital_1,Provisões,Passivos sobre Ativos Não-Correntes a Venda e Descontinuados,Lucros e Receitas a Apropriar,Participação dos Acionistas Não Controladores,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital,Receita Líquida de Vendas e/ou Serviços,Custo de Bens e/ou Serviços Vendidos,Resultado Bruto,Despesas Com Vendas,Despesas Gerais e Administrativas,Perdas pela Não Recuperabilidade de Ativos,Outras Receitas Operacionais,Outras Despesas Operacionais,Resultado da Equivalência Patrimonial,Financeiras,Receitas Financeiras,Despesas Financeiras,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Lucro/Prejuízo do Período,Adj Close,IBOV
2020-03-31,9.716450e+08,1.635620e+08,8.038200e+07,3345999.872,1.586600e+07,3.123600e+07,0,1.315000e+07,0.0,1.958200e+07,1.197740e+08,0.0,0.0,2.170000e+05,1.200200e+07,0,0.000000e+00,6.553800e+07,0.0,0.000000e+00,1.997300e+07,5.898140e+08,7.852200e+07,0,9.716450e+08,1.348370e+08,6152000.000,3.026200e+07,998000.000,5.923400e+07,0.0,1808000.0,1.900900e+07,4052999.936,1.332100e+07,6.018830e+08,4.046820e+08,0.0,1992999.936,8.820000e+05,0,1.943260e+08,0,0,3096000.000,2.318290e+08,2.054320e+08,2448999.936,0,7.630400e+07,0.000000e+00,0.0,0.0,-5.235600e+07,0,7.546900e+07,-4.385400e+07,3.161500e+07,-5913999.872,-1820000.000,0,0,-6.788200e+07,-1439000.064,-2.117800e+07,798000.000,-2.197600e+07,-6.661800e+07,-597000.000,1.749100e+07,-4.852300e+07,11.370116,73020.0
2019-09-30,9.244650e+08,1.476010e+08,5.488200e+07,5426999.808,1.749500e+07,3.158300e+07,0,1.078800e+07,0.0,2.742600e+07,7.463600e+07,0.0,0.0,1.980000e+05,1.027900e+07,0,0.000000e+00,2.278800e+07,0.0,0.000000e+00,2.357900e+07,6.687420e+08,9.907000e+06,0,9.244650e+08,1.336600e+08,7369999.872,2.435700e+07,574000.000,5.486100e+07,0.0,2472000.0,2.269500e+07,3147000.064,1.818400e+07,4.810520e+08,3.195200e+08,0.0,2063000.064,1.073500e+07,0,1.487340e+08,0,0,3500999.936,3.062520e+08,2.054320e+08,2455000.064,0,1.234320e+08,0.000000e+00,0.0,0.0,-2.506700e+07,0,7.705100e+07,-4.704500e+07,3.000600e+07,-4968000.000,-2012000.000,0,0,-9.169000e+06,446000.000,-1.087400e+07,1344000.000,-1.221800e+07,3.429000e+06,758000.000,-4.696000e+06,9.087000e+06,21.939379,104745.0
2017-06-30,8.080540e+08,1.424350e+08,7.797000e+07,3316999.936,1.447700e+07,2.662100e+07,0,8.361000e+06,0.0,1.168900e+07,6.752000e+07,0.0,715000.0,1.742400e+07,0.000000e+00,0,2.003400e+07,0.000000e+00,0.0,2.934700e+07,1.230700e+07,5.752420e+08,1.055000e+07,0,8.080540e+08,6.996800e+07,5216999.936,1.700100e+07,2161999.872,2.598500e+07,0.0,0.0,1.623700e+07,2664999.936,7.010000e+05,4.769420e+08,3.506020e+08,0.0,2921999.872,3.526000e+06,0,1.198920e+08,0,0,2508999.936,2.586350e+08,2.054320e+08,1046000.000,0,7.780000e+07,4.770000e+06,0.0,0.0,-3.041300e+07,0,6.699600e+07,-4.562700e+07,2.136900e+07,-3888999.936,-2220999.936,0,0,-2

In [ ]:
# transform indicators into percentages
# cotacao%tri - cotacao tri seguinte / cotacao tri

for company in fundamentals:
    indicator = fundamentos[company]
    indicator = indicator.sort_index()
    
    # get next quote
    indicator['Adj Close'] = indicator['Adj Close'].shift( -1 ) / indicator['Adj Close'] - 1
    indicator['IBOV'] = indicator['IBOV'].shift( -1 ) / indicator['IBOV'] - 1
    indicator['Resultado'] = indicator['Adj Close'] - indicator['IBOV']

    condicoes = [
        ( indicator['Resultado'] > 0 ),
        ( indicator['Resultado'] < 0 ) & ( indicator['Resultado'] >= -0.02 ),
        (indicator['Resultado'] < -0.02)
    ]

    valores = [2, 1, 0]
    indicator['Decisao'] = np.select( condicoes, valores )
        
    fundamentos[company] = indicator

### 0.2.4. Making everything a single dataframe

In [21]:
copy_fundamentals = fundamentals.copy()

In [30]:
df = pd.DataFrame()

for company in copy_fundamentals:
    pd.concat( [df, copy_fundamentals[company]], axis=0, ignore_index=True )

df.to_csv('Fundamentals.csv' )
df.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 80 elements

In [34]:
copy_fundamentals['PETR4']

,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,Outros Ativos Circulantes,...,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período,Adj Close
PETR4,,,,,,,,,,,,,,,,,,,,,
2020-03-31,9.71645e+08,1.63562e+08,8.0382e+07,3.346e+06,1.5866e+07,3.1236e+07,0,1.315e+07,0,1.9582e+07,...,NaN,NaN,-6.6618e+07,-597000,1.7491e+07,NaN,NaN,1.201e+06,-4.8523e+07,11.370116
2019-09-30,9.24465e+08,1.47601e+08,5.4882e+07,5.427e+06,1.7495e+07,3.1583e+07,0,1.0788e+07,0,2.7426e+07,...,NaN,NaN,3.429e+06,758000,-4696000,NaN,NaN,247000,9.087e+06,21.939379
2017-06-30,8.08054e+08,1.42435e+08,7.797e+07,3.317e+06,1.4477e+07,2.6621e+07,0,8.361e+06,0,1.1689e+07,...,NaN,NaN,6.77e+06,-2.573e+06,-3.905e+06,NaN,NaN,24000,316000,9.327338
2017-03-31,7.88046e+08,1.34058e+08,6.0874e+07,2.909e+06,1.4042e+07,2.6172e+07,0,8.167e+06,0,2.1894e+07,...,NaN,NaN,7.127e+06,-826000,-1494000,NaN,NaN,-358000,4.449e+06,10.925880
2016-09-30,8.03206e+08,1.44753e+08,7.006e+07,2.542e+06,1.6953e+07,2.7627e+07,0,8.709e+06,0,1.8862e+07,...,NaN,NaN,-1.7294e+07,-1009000,1980000,NaN,NaN,-135000,-1.6458e+07,10.232175
2016-06-30,8.18332e+08,1.32625e+08,6.294e+07,2.43e+06,1.7047e+07,2.8508e+07,0,9.285e+06,0,1.2415e+07,...,NaN,NaN,1.521e+06,-1.911e+06,1.289e+06,NaN,NaN,-529000,370000,7.102953
2016-03-31,8.5916e+08,1.46243e+08,7.7778e+07,2.729e+06,1.8865e+07,2.9098e+07,0,1.0612e+07,0,7.161e+06,...,NaN,NaN,-157000,-1.637e+06,1.413e+06,NaN,NaN,-865000,-1246000,6.265982
2015-09-30,9.31562e+08,1.7638e+08,9.987e+07,4.379e+06,2.1155e+07,3.2585e+07,0,1.0172e+07,0,8.219e+06,...,NaN,NaN,-5.199e+06,-814000,988000,NaN,NaN,1266000,-3.759e+06,5.459170
2015-06-30,8.59299e+08,1.6038e+08,8.1166e+07,1.0478e+07,2.005e+07,3.3771e+07,0,9.927e+06,0,4.988e+06,...,NaN,NaN,3.581e+06,-905000,-1768000,NaN,NaN,-377000,531000,9.568627


# 1.0. Data Description

# 2.0. Feature Engineering

# 3.0. Data Filtering

# 4.0.Exploratory Data Analysis

# 5.0. Data Preparation

# 6.0. Feature Selection

# 7.0. Machine Learning Modeling

# 8.0. Hyperparameter Fine Tuning

# 9.0. Machine Learning Performance

# 10.0. Deploy Model to Production

# 11.0. Conclusions